# Usage example notebook

In [ ]:
from langchain.prompts.chat import ChatPromptTemplate
from langserve import RemoteRunnable

llm = RemoteRunnable("http://localhost:8000/")

Let's create a prompt composed of a system message and a human message.

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Eres un asistente genial, a la vez que conciso.",),
        ("human", "Proporciona ideas de premios para un tech challenge que estoy organizando, cada equipo se compone de 5 personas y solo hay un equipo ganador."),
    ]
).format_messages()

As with regular runnables, invoke (sync/async), batch (sync/async) and stream (sync/async) variants are available by default

In [ ]:
llm.invoke(prompt)

In [ ]:
await llm.ainvoke(prompt)

In [ ]:
llm.batch([prompt, prompt])

In [ ]:
await llm.abatch([prompt, prompt])

In [ ]:
for chunk in llm.stream(prompt):
    print(chunk, end="", flush=True)

In [ ]:
async for chunk in llm.astream(prompt):
    print(chunk.content, end="", flush=True)

Complex example

In [ ]:
from langchain.schema.runnable import RunnablePassthrough

comedian_chain = (
    ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You are a comedian that sometimes tells funny jokes and other times you just state facts that are not funny. Please either tell a joke or state fact now but only output one.",
            ),
        ]
    )
    | llm
)

joke_classifier_chain = (
    ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "Please determine if the joke is funny. Say `funny` if it's funny and `not funny` if not funny. Then repeat the first five words of the joke for reference...",
            ),
            ("human", "{joke}"),
        ]
    )
    | llm
)


chain = {"joke": comedian_chain} | RunnablePassthrough.assign(
    classification=joke_classifier_chain
)

chain.invoke({})